In [2]:
from __future__ import print_function, division
import pydicom
import torch
import os
import sys
import pandas as pd
import numpy as np
import pydicom as DCM
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
from pathlib import Path
from sklearn.preprocessing import LabelEncoder


In [ ]:
# class MnistBags(data_utils.Dataset):
#     def __init__(self, target_number, mean_bag_length, var_bag_length, num_bag=250, seed=1, train=True):
#         self.target_number = target_number
#         self.mean_bag_length = mean_bag_length
#         self.var_bag_length = var_bag_length
#         self.num_bag = num_bag
#         self.train = train

#         self.r = np.random.RandomState(seed)

#         self.num_in_train = 60000
#         self.num_in_test = 10000

#         if self.train:
#             self.train_bags_list, self.train_labels_list = self._create_bags()
#         else:
#             self.test_bags_list, self.test_labels_list = self._create_bags()

#     def _create_bags(self):
#         if self.train:
#             loader = data_utils.DataLoader(datasets.MNIST('../datasets',
#                                                           train=True,
#                                                           download=True,
#                                                           transform=transforms.Compose([
#                                                               transforms.ToTensor(),
#                                                               transforms.Normalize((0.1307,), (0.3081,))])),
#                                            batch_size=self.num_in_train,
#                                            shuffle=False)
#         else:
#             loader = data_utils.DataLoader(datasets.MNIST('../datasets',
#                                                           train=False,
#                                                           download=True,
#                                                           transform=transforms.Compose([
#                                                               transforms.ToTensor(),
#                                                               transforms.Normalize((0.1307,), (0.3081,))])),
#                                            batch_size=self.num_in_test,
#                                            shuffle=False)

#         for (batch_data, batch_labels) in loader:
#             all_imgs = batch_data
#             all_labels = batch_labels
        
        
#         bags_list = []
#         labels_list = []

#         for i in range(self.num_bag):
#             bag_length = np.int(self.r.normal(self.mean_bag_length, self.var_bag_length, 1))
#             if bag_length < 1:
#                 bag_length = 1

#             if self.train:
#                 indices = torch.LongTensor(self.r.randint(0, self.num_in_train, bag_length))
#             else:
#                 indices = torch.LongTensor(self.r.randint(0, self.num_in_test, bag_length))

#             labels_in_bag = all_labels[indices]
#             labels_in_bag = labels_in_bag == self.target_number

#             bags_list.append(all_imgs[indices])
#             labels_list.append(labels_in_bag)

#         return bags_list, labels_list

#     def __len__(self):
#         if self.train:
#             return len(self.train_labels_list)
#         else:
#             return len(self.test_labels_list)

#     def __getitem__(self, index):
#         if self.train:
#             bag = self.train_bags_list[index]
#             label = [max(self.train_labels_list[index]), self.train_labels_list[index]]
#         else:
#             bag = self.test_bags_list[index]
#             label = [max(self.test_labels_list[index]), self.test_labels_list[index]]

#         return bag, label

'/home/himanshu'

In [3]:
def createTrainFrame(homedir):
    
    train_mass_csv = pd.read_csv(homedir+"/CuratedDDSM/Train/mass_case_description_train_set.csv")
    train_calc_csv = pd.read_csv(homedir+"/CuratedDDSM/Train/calc_case_description_train_set.csv")
    train_calc_csv = train_calc_csv.rename(columns={'breast density': 'breast_density'})
    train_calc_csv['image file path'] = 'Calc/CBIS-DDSM/'+ train_calc_csv['image file path']
    train_calc_csv['ROI mask file path'] ='CalcROI/CBIS-DDSM/' + train_calc_csv['ROI mask file path']
    train_calc_csv['cropped image file path'] ='CalcROI/CBIS-DDSM/' + train_calc_csv['cropped image file path']
    train_mass_csv['image file path'] = 'Mass/CBIS-DDSM/' + train_mass_csv['image file path']
    train_mass_csv['ROI mask file path'] ='MassROI/CBIS-DDSM/' + train_mass_csv['ROI mask file path']
    train_mass_csv['cropped image file path'] ='MassROI/CBIS-DDSM/'+ train_mass_csv['cropped image file path']
    common_col = list(set(train_calc_csv.columns) & set(train_mass_csv.columns))
    train = pd.concat([train_mass_csv[common_col],train_calc_csv[common_col]], ignore_index=True,sort='False')
    #train = train_mass_csv
    train['image file path'] = homedir+'/CuratedDDSM/Train/'+train['image file path'] 
    train['ROI mask file path'] = homedir+'/CuratedDDSM/Train/'+train['ROI mask file path']
    train['cropped image file path'] = homedir+'/CuratedDDSM/Train/'+train['cropped image file path']
    train=train[train.pathology!='BENIGN_WITHOUT_CALLBACK']
    train['pathology_class'] = LabelEncoder().fit_transform(train['pathology'])
    train =  pd.concat([train,pd.get_dummies(train['pathology'])],axis=1)
    
    ''' proper ROI mask present check'''
    
    train['image file path']=train['image file path'].str.replace('\n',"")
    train['ROI mask file path']=train['ROI mask file path'].str.replace('\n',"")
    train['cropped image file path']=train['cropped image file path'].str.replace('\n','')
    
    notfound = []
    for index, row in train.iterrows():
        if(os.path.isfile(row['cropped image file path'])==False
           or os.path.isfile(row['ROI mask file path'])==False or os.path.isfile(row['image file path'])==False):
            notfound.append(index)
    
    train.drop(train.index[notfound], inplace=True)
    
    train = train.reset_index(drop=True)
    
    
    
    
    
    
    
    return train

In [4]:
train = createTrainFrame('/media/erh/BEFCFF6AFCFF1B7B/dataset/')

In [5]:
samplepath =  train['image file path'][14]

In [6]:
img  = DCM.dcmread(samplepath).pixel_array
img.shape == img.shape

True

In [5]:
okcount=0
count=0
deleteIndex = []
for index, row in train.iterrows():
    mask = DCM.dcmread(row['ROI mask file path']).pixel_array 
    crop = DCM.dcmread(row['cropped image file path']).pixel_array
    img = DCM.dcmread(row['image file path']).pixel_array
    if(img.shape==mask.shape):
        okcount=okcount+1
    else:
        if(img.shape==crop.shape):
            train.loc[index,['cropped image file path','ROI mask file path']] = train.loc[index,['ROI mask file path','cropped image file path']].values
        else:
            deleteIndex.append(index)

In [12]:
okcount

606

In [ ]:
okcount=0
count=0
for index, row in train.iterrows():
    mask = DCM.dcmread(row['ROI mask file path']).pixel_array 
    crop = DCM.dcmread(row['cropped image file path']).pixel_array
    img = DCM.dcmread(row['image file path']).pixel_array
    if(img.shape==mask.shape):
        okcount=okcount+1
    else:
        #train.loc[index,['cropped image file path','ROI mask file path']] = train.loc[index,['ROI mask file path','cropped image file path']].values
        print(img.shape,mask.shape,crop.shape)

(5386, 3601) (4686, 3133) (275, 318)
(5386, 3466) (4686, 3016) (267, 346)
(5386, 3391) (4686, 2951) (344, 319)
(5386, 3436) (4686, 2990) (306, 337)
(5386, 3691) (4686, 3212) (539, 698)
(5221, 3391) (4543, 2951) (428, 316)
(5296, 3061) (4608, 2664) (369, 440)
(5296, 3436) (4608, 2990) (374, 399)
(5026, 2131) (4373, 1854) (517, 553)
(4966, 3061) (4321, 2664) (304, 318)
(5326, 3421) (4634, 2977) (363, 445)
(5356, 2581) (4660, 2246) (350, 356)
(5386, 3061) (4686, 2664) (347, 388)
(5296, 2566) (4608, 2233) (420, 357)
(5386, 2836) (4686, 2468) (372, 360)
(5011, 2596) (4360, 2259) (554, 563)
(5116, 2371) (4451, 2063) (431, 428)
(5221, 3121) (4543, 2716) (385, 374)
(5386, 3436) (4686, 2990) (535, 500)
(5356, 2851) (4660, 2481) (313, 318)
(5266, 2761) (4582, 2403) (337, 340)


In [ ]:
okcount

In [ ]:
count=0
notfound=[]
for index, row in train.iterrows():
    if(os.path.isfile(row['cropped image file path'])):
        notfound.append(index)
        count=count+1

In [ ]:
count

In [ ]:
notfound

In [ ]:
train['ROI mask file path'][0]

In [ ]:
print(len(train))

In [ ]:
for idx in range(len(train)):
    """ Clean the data,"""